In [7]:
import sys

sys.path.append('C:\\Users\\admin\\AppData\\Local\\conda\\conda\\envs\\dl_env\\Lib\\site-packages')

import re
import time
import math
import model
import heapq
import numpy
import random
import string
import pickle
import theano
import sunfish
import traceback
import chess, chess.pgn
import theano.tensor as T
from IPython.display import display, HTML, clear_output

diff10 = 'Model/2600model.pickle'
diff9 = 'Model/2400-2599model.pickle'
diff8 = 'Model/2200-2399model.pickle'
diff7 = 'Model/2000-2199model.pickle'
diff6 = 'Model/1800-1999model.pickle'
diff5 = 'Model/1600-1799model.pickle'
diff4 = 'Model/1400-1599model.pickle'
diff3 = 'Model/1200-1399model.pickle'
diff2 = 'Model/1000-1199model.pickle'
diff1 = 'Model/800-999model.pickle'
diff0 = 'Model/600-799model.pickle'

strip_whitespace = re.compile(r"\s+")
translate_pieces = str.maketrans(".pnbrqkPNBRQK", "\x00" + "\x01\x02\x03\x04\x05\x06" + "\x08\x09\x0a\x0b\x0c\x0d")

def display_board(board, use_svg):
    """
    Display chess board using svg rendering.
    """
    if use_svg:
        return board._repr_svg_()
    else:
        return "<pre>" + str(board) + "</pre>"
    
def who(player):
    """
    Return whose turn it is for display purposes.
    """
    return "White" if player == chess.WHITE else "Black"  

def play_game(player1, player2, visual="svg", pause=0.1):
    """
    player1, player2: functions that make moves, return uci move
    visual: "simple" | "svg" | None
    """
    use_svg = (visual == "svg")
    board = chess.Board()
    gn_current = chess.pgn.Game()
    
    try:
        while not board.is_game_over():
            if board.turn == chess.WHITE:
                gn_current = player1.calc_move(gn_current)
                uci = player1.get_ai_move()
            else:
                uci = player2(board)
                gn_current = get_game(gn_current, uci)
                
            name = who(board.turn)
            board.push_uci(uci)
            board_stop = display_board(board, use_svg)
            html = "<b>Move %s %s, Play '%s':</b><br/>%s" % (
                       len(board.move_stack), name, uci, board_stop)
            if visual is not None:
                if visual == "svg":
                    clear_output(wait=True)
                display(HTML(html))
                if visual == "svg":
                    time.sleep(pause)
    except KeyboardInterrupt:
        msg = "Game interrupted!"
        return (None, msg, board)
    result = None
    if board.is_checkmate():
        msg = "Checkmate: " + who(not board.turn) + " WINS!"
        result = not board.turn
    elif board.is_stalemate():
        msg = "Draw: stalemate"
    elif board.is_fivefold_repetition():
        msg = "Draw: 5-fold repetition"
    elif board.is_insufficient_material():
        msg = "Draw: insufficient material"
    elif board.can_claim_draw():
        msg = "Draw: claim"
    if visual is not None:
        print(msg)
    return (result, msg, board)    
    
def get_model_from_pickle(fn):
    """
    Open pickle file and load the data 
    Using model file functions
    """
    f = open(fn, "rb")
    Ws, bs = pickle.load(f, encoding='latin1')
    
    Ws_s, bs_s = model.get_parameters(Ws=Ws, bs=bs)
    x, p = model.get_model(Ws_s, bs_s)
    
    predict = theano.function(
        inputs=[x],
        outputs=p)

    return predict


def sf2array(pos, flip):
    # Create a numpy array from a sunfish representation
    pos = strip_whitespace.sub('', pos.board) # should be 64 characters now
    pos = pos.translate(translate_pieces)
    m = numpy.fromstring(pos, dtype=numpy.int8)
    if flip:
        m = numpy.fliplr(m.reshape(8, 8)).reshape(64)
    return m

CHECKMATE_SCORE = 1e6

def negamax(pos, depth, alpha, beta, color, func):
    moves = []
    X = []
    pos_children = []
    for move in pos.gen_moves():
        pos_child = pos.move(move)
        moves.append(move)
        X.append(sf2array(pos_child, flip=(color==1)))
        pos_children.append(pos_child)

    if len(X) == 0:
        return Exception('eh?')

    # Use model to predict scores
    scores = func(X)

    for i, pos_child in enumerate(pos_children):
        if pos_child.board.find('K') == -1:
            scores[i] = CHECKMATE_SCORE

    child_nodes = sorted(zip(scores, moves), reverse=True)

    best_value = float('-inf')
    best_move = None
    
    for score, move in child_nodes:
        if depth == 1 or score == CHECKMATE_SCORE:
            value = score
        else:
            pos_child = pos.move(move)
            neg_value, _ = negamax(pos_child, depth-1, -beta, -alpha, -color, func)
            value = -neg_value

        if value > best_value:
            best_value = value
            best_move = move

        if value > alpha:
            alpha = value

        if alpha > beta:
            break

    return best_value, best_move


def create_move(board, crdn):
    # workaround for pawn promotions
    move = chess.Move.from_uci(crdn)
    if board.piece_at(move.from_square).piece_type == chess.PAWN:
        if int(move.to_square/8) in [0, 7]:
            move.promotion = chess.QUEEN # always promote to queen
    return move

class Player(object):
    def move(self, gn_current):
        raise NotImplementedError()

class Computer(Player):
    def __init__(self, func, maxd=5):
        self._func = func
        self._pos = sunfish.Position(sunfish.initial, 0, (True,True), (True,True), 0, 0)
        self._maxd = maxd

    def calc_move(self, gn_current):
        """
        Calculate the best move using sunfish engine/negamax function
        Returns pgn game instance
        """
        #assert(gn_current.board().turn == True)

        if gn_current.move is not None:
            crdn = str(gn_current.move)
            move = (119 - sunfish.parse(crdn[0:2]), 119 - sunfish.parse(crdn[2:4]))
            self._pos = self._pos.move(move)

        alpha = float('-inf')
        beta = float('inf')

        depth = self._maxd
        t0 = time.time()
        best_value, best_move = negamax(self._pos, depth, alpha, beta, 1, self._func)
        crdn = sunfish.render(best_move[0]) + sunfish.render(best_move[1])

        self._pos = self._pos.move(best_move)
        crdn = sunfish.render(best_move[0]) + sunfish.render(best_move[1])
        self.move = create_move(gn_current.board(), crdn)
        
        gn_new = chess.pgn.GameNode()
        gn_new.parent = gn_current
        gn_new.move = self.move

        return gn_new
        
    def get_ai_move(self):
        """
        Return the uic value of move made
        """
        move = self.move.uci()
        return move
    

def human_player(board):
    """
    Calls get move, makes sure move is legal, returns uci value
    """
    uci = get_move("%s's move [q to quit]> " % who(board.turn))
    legal_uci_moves = [move.uci() for move in board.legal_moves]
    while uci not in legal_uci_moves:
        print("Legal moves: " + (",".join(sorted(legal_uci_moves))))
        uci = get_move("%s's move[q to quit]> " % who(board.turn))
    return uci

def get_move(prompt):
    """
    Gets move from user, keyboard entry
    """
    uci = input(prompt)
    if uci and uci[0] == "q":
        raise KeyboardInterrupt()
    try:
        chess.Move.from_uci(uci)
    except:
        uci = None
    return uci

def get_game(gn_current, uci):
    """
    Sets game variables after human makes a move,
    for the AI to reference
    """
    bb = gn_current.board()
    move = chess.Move.from_uci(uci)
    gn_new = chess.pgn.GameNode()
    gn_new.parent = gn_current
    gn_new.move = move
    return gn_new


def game(func):
    """
    Computer class instance with max depth,
    Calls play game function
    """
    maxd = random.randint(1, 2) 

    ai_player = Computer(func, maxd=maxd)

    play_game(ai_player, human_player)

            
def play():
    """
    Gets model and passes it to game
    """
    func = get_model_from_pickle(diff1)
    while True:
        game(func)
        
        
if __name__ == '__main__':
    play()


ValueError: illegal uci: 'f3d4' in rnbqkbnr/1ppp1ppp/7B/8/4P3/2NP1N2/P1P2PPP/3qKB1R w Kkq - 0 8